In [2]:
import torch 
import math 
import numpy as np
import copy
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import ast 
from numpy import load
import torch.nn as nn
import random
import time
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
import collections
#import helper
import numpy as np
#import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [6]:
def read_sentences(file_path):
	sentences = []

	with open(file_path, 'r') as reader:
		for s in reader:
			sentences.append(s.strip())

	return sentences

In [457]:
sanskrit_vectors=read_sentences("/home/deepak/Documents/Atharva/embeddings/vectors.txt")

In [458]:
sanskrit_glove={}
for key in sanskrit_vectors:
    sanskrit_glove[key.split()[0]]=np.array((key.split()[1:])).astype(float)

In [459]:
from keras.preprocessing.text import Tokenizer

st = Tokenizer()
# fit the tokenizer on the documents
st.fit_on_texts(sanskrit_data)
encoded_sanskrit_train_data = st.texts_to_sequences(sanskrit_data)

In [460]:
from keras.preprocessing.text import Tokenizer
et = Tokenizer()
# fit the tokenizer on the documents
et.fit_on_texts(english_data)
encoded_english_train_data = et.texts_to_sequences(english_data)

In [461]:
padded_sanskrit_train_data=pad_sequences(encoded_sanskrit_train_data,  padding='post')
padded_english_train_data=pad_sequences(encoded_english_train_data,  padding='post')


In [462]:
matrix_len = len(t.word_index)
weights_matrix = np.zeros((matrix_len+1, 100))
words_found = 0

for word in t.word_index.keys():
    try: 
        weights_matrix[t.word_index[word]] = sanskrit_glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[t.word_index[word]] = np.random.normal(scale=0.6, size=(100, ))

In [463]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [7]:
sanskrit_data=read_sentences("/home/deepak/Downloads/Sanskrit.txt")
english_data=read_sentences("/home/deepak/Downloads/English.txt")

In [3]:
import pandas as pd
Dictionary = pd.read_csv("/home/deepak/Downloads/Dictionary.csv")
Atharva = pd.read_csv("/home/deepak/Downloads/dataset_atharva.csv")
Rigved = pd.read_csv("/home/deepak/Downloads/dataset_rig.csv")
df=pd.concat([Dictionary,Atharva,Rigved],ignore_index=True)

In [4]:
sanskrit_data_length=0
english_data_length=0
for i in range(len(df)):
    if len(df['Sanskrit'][i].split())>sanskrit_data_length:
        sanskrit_data_length=len(df['Sanskrit'][i].split())
for i in range(len(df)):
    if len(df['English'][i].split())>english_data_length:
        english_data_length=len(df['English'][i].split())

In [5]:
english_data_length,sanskrit_data_length

(499, 196)

In [6]:
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset


SN_TEXT = Field(fix_length=sanskrit_data_length)
EN_TEXT = Field( fix_length=english_data_length ,init_token = "<sos>", eos_token = "<eos>")

In [7]:
import pandas as pd
raw_data = {'Sanskrit' : [line for line in sanskrit_data], 'English': [line for line in english_data]}
df = pd.DataFrame(raw_data, columns=["Sanskrit", "English"])

NameError: name 'sanskrit_data' is not defined

In [12]:
#df['eng_len'] = df['English'].str.count(' ')
#df['sns_len'] = df['Sanskrit'].str.count(' ')
#df = df.query('sns_len < 80 & eng_len < 80')
#df = df.query('eng_len < sns_len * 1.5 & eng_len * 1.5 > sns_len')

In [7]:
df

,Sanskrit,English
0,aktu,tinge
1,aktu,ray
2,aktu,light
3,aktu,darkness
4,atya,steed
...,...,...
15292,viṣṇu̍r i̱tthā pa̍ra̱mam a̍sya vi̱dvāñ jā̱to b...,here being manifested lofty viṣṇu full wise pr...
15293,ata̍ u tvā pitu̱bhṛto̱ jani̍trīr annā̱vṛdha̱m ...,thence bearing food the mothers come to meet t...
15294,hotā̍raṁ ci̱trara̍tham adhva̱rasya̍ ya̱jñasya̍...,priest of the holy rite with car that glitters...
15295,sa tu vastrā̱ṇy adha̱ peśa̍nāni̱ vasā̍no a̱gni...,so agni stands on earth's most central station...


In [8]:
from sklearn.model_selection import train_test_split
# create train and validation set 
train, val = train_test_split(df, test_size=0.1)
train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)

In [9]:
data_fields = [('Sanskrit', SN_TEXT), ('English', EN_TEXT)]
train,val = TabularDataset.splits(path='./', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

In [10]:
SN_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)

In [11]:
train_iter = BucketIterator(train, batch_size=10, \
sort_key=lambda x: len(x.English), shuffle=True)

In [12]:
import torch.nn as nn

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [13]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 499):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False).cuda(1)
        return x

In [14]:
batch = next(iter(train_iter))
input_seq = batch.Sanskrit.transpose(0,1)
input_pad = SN_TEXT.vocab.stoi['<pad>']
# creates mask with 0s wherever there is padding in the input
input_msk = (input_seq != input_pad).unsqueeze(1)

In [15]:
from torch.autograd import Variable
import torch
import numpy as np
# create mask as before
target_seq = batch.English.transpose(0,1)
target_pad = EN_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) 
# get seq_len for matrix
nopeak_mask = np.triu(np.ones((1, size, size)),k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
target_msk = target_msk & nopeak_mask

In [16]:
input_msk = input_msk.cuda(1)
target_msk = target_msk.cuda(1)

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [18]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
        
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

In [19]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [20]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [21]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda(1)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
        # We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [22]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [23]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [24]:

def nopeak_mask(size,cuda_enabled):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  torch.autograd.Variable(torch.from_numpy(np_mask) == 0)

    if cuda_enabled:
      np_mask = np_mask.cuda(1)
    return np_mask

def create_masks(src, trg):
    src_mask = (src != 0).unsqueeze(-2)
    if trg is not None:
        trg_mask = (trg != 0).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        # print("Sequence lenght in mask ",size)
        np_mask = nopeak_mask(size,True)
        # print(np_mask.shape,trg_mask.shape)
        if trg.is_cuda:
            np_mask.cuda(1)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [10]:
d_model = 80
heads = 8
N = 6
src_vocab = len(SN_TEXT.vocab)
trg_vocab = len(EN_TEXT.vocab)
model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    print(model.parameters())
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

NameError: name 'SN_TEXT' is not defined

In [90]:
def train_model(epochs, print_every=100):
    model.cuda(1)
    model.train()

    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iter):            
            src = batch.Sanskrit.transpose(0,1)
            trg = batch.English.transpose(0,1)            
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            if torch.cuda.is_available():
                src = src.cuda(1)
                trg_input = trg_input.cuda(1)
            
            #print(src.is_cuda)    
            #print(trg_input.is_cuda)
            #print(input_msk.is_cuda)
            #print(target_msk.is_cuda)
            src_mask, trg_mask = create_masks(src, trg_input)
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),targets.cuda(1), ignore_index=target_pad)            
            loss.backward()
            optim.step()
            
            print(loss.data)
            total_loss += loss.data
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg, time.time() - temp,print_every))
                total_loss = 0
                temp = time.time()

In [88]:
train_model(50, print_every=1)

RuntimeError: shape '[10, -1, 8, 6]' is invalid for input of size 98000

In [37]:
def _split_tokenizer(x): 
    return x.split()


In [20]:
torch.tensor([[4,5,3],[6,7,3]])[:, 1:]

tensor([[5, 3],
        [7, 3]])

In [11]:
import tensorflow as tf

tf.constant([[2,3],[3,4]])[:, 1:]


<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [4]], dtype=int32)>

In [17]:
tf.transpose([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]], perm=[0, 2, 1, 3])
tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))

InvalidArgumentError: transpose expects a vector of size 2. But input(1) is a vector of size 4 [Op:Transpose]

In [38]:
def translate(model, src, max_len = 499, custom_sentence=False):
    
    model.eval()
    if custom_sentence == True:
        src = _split_tokenizer(src)
        sentence= Variable(torch.LongTensor([[SN_TEXT.vocab.stoi[tok] for tok in src]])).cuda(1)
        print(sentence)
    src_mask = (sentence != 1).unsqueeze(-2)
    e_outputs = model.encoder(sentence, src_mask)
    
    outputs = torch.zeros(max_len).type_as(sentence.data)
    outputs[0] = torch.LongTensor([EN_TEXT.vocab.stoi['<sos>']])
    for i in range(1, max_len):                
        trg_mask = np.triu(np.ones((1, i, i)),k=1).astype('uint8')
        trg_mask= Variable(torch.from_numpy(trg_mask) == 0).cuda(1)
        out = model.out(model.decoder(outputs[:i].unsqueeze(0),e_outputs, src_mask, trg_mask))
        out = F.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)
        
        outputs[i] = ix[0][0]
        if ix[0][0] == EN_TEXT.vocab.stoi['<eos>']:
            break
    return ' '.join([EN_TEXT.vocab.itos[ix] for ix in outputs[:i]])

In [49]:
validation_data=pd.read_csv('val.csv')
validation_data

,Sanskrit,English
0,ti̱sro vāca̱ḥ pra va̍da̱ jyoti̍ragrā̱ yā e̱tad...,speak forth three words the words which light ...
1,gṛhāṇa grāvāṇau sakṛtau vīra hasta ā te devā ...,grasp with thy hand o man the well-formed pres...
2,hi̱meva̍ pa̱rṇā mu̍ṣi̱tā vanā̍ni̱ bṛha̱spati̍n...,as trees for foliage robbed by winter vala mou...
3,ma̱rutsto̍trasya vṛ̱jana̍sya go̱pā va̱yam indr...,guards of the camp whose praisers are the maru...
4,ya enaṃ hanti mṛduṃ manyamāno devapīyur dhana...,whoever smites him deeming him a weakling-blas...
...,...,...
1525,viśva̍smā a̱gnim bhuva̍nāya de̱vā vai̍śvāna̱ra...,for all the world of life the gods made agni v...
1526,a̱sme rāyo̍ di̱vedi̍ve̱ saṁ ca̍rantu puru̱spṛh...,so unto us day after day may riches craved by ...
1527,a̱ryo vi̱śāṁ gā̱tur e̍ti̱ pra yad āna̍ḍ di̱vo ...,kind furtherer of men he comes when he hath re...
1528,pra va̱ḥ pānta̍ṁ raghumanya̱vo 'ndho̍ ya̱jñaṁ ...,say bringing sacrifice to bounteous rudra this...


In [50]:
translated_validation=[]
for i in range(len(validation_data)):
    translated_validation.append(translate(model,src=validation_data['Sanskrit'][i],custom_sentence=True))


612,    13,   769, 47827,  3391,  9878,
          3154, 30378,  1165,   318, 10850,  2841,  2275]], device='cuda:1')
tensor([[20405, 36015, 41183, 63808, 13074, 11276,   315,  5418, 29183, 47379,
          2329, 41387, 22305, 25226]], device='cuda:1')
tensor([[   10, 28156,  8379, 41543, 48283,  2610, 10893,  1277,    30,  5501,
          2512,   103, 36732, 23115, 14967,  8049]], device='cuda:1')
tensor([[14061,  7630,  1070,  9350,   815,   151, 67221,  3219,  1937,   868,
         18958,  2793, 16984, 13064, 53426,   253]], device='cuda:1')
tensor([[ 3710,   170,  2354,     3,  2862,  4277, 47061, 25649,     4, 58251,
         47662,  2523,   214, 41500,   162,   466,  1400]], device='cuda:1')
tensor([[    5, 48000, 12384,  2019,  1415,  3321, 28996,  2469, 37544, 48442,
           592, 13525]], device='cuda:1')
tensor([[   50,  4391, 64809,  2072, 65934,    16,  1238,   210,   228,  6703]],
       device='cuda:1')
tensor([[ 7776,   132,    29,   182, 43262,   200,  1439,  2491,  13

In [51]:
raw_data = {'English': translated_validation}
df = pd.DataFrame(raw_data, columns=["English"])
df.to_csv("translated_validation.csv", index=False)

In [55]:
import sacrebleu
test = pd.read_csv('translated_validation.csv')

In [85]:
validation_data['Sanskrit'][1]

' gṛhāṇa grāvāṇau sakṛtau vīra hasta ā te devā yajñiyā yajñam aguḥ  trayo varā yatamāṃs tvaṃ vṛṇīṣe tās te samṛddhīr iha rādhayāmi'

In [84]:
translate(model,src=validation_data['Sanskrit'][1],custom_sentence=True)

tensor([[15762, 15718, 51563,  3762, 15831,     2,     5,    53,  3252,  1274,
          9099,  4390, 12148, 64214,    80, 63777,  1116,     5, 51811,   178,
         50862]], device='cuda:1')


'<sos> gods with this thy worshipper is life and hence in which we dwell together also in thy hand and hence far as thy mind etc'

In [83]:
test['English'][1],validation_data['English'][1]

(' gods with this thy worshipper is life and hence in which we dwell together also in thy hand and hence far as thy mind etc',
 'grasp with thy hand o man the well-formed press-stones the holy gods have come unto thy worship three wishes of thy heart which thou electest these happy gains for thee i here make ready')

In [82]:
# all words different
from nltk.translate.bleu_score import sentence_bleu

i=1
score = sentence_bleu(validation_data['English'][i].split(),test['English'][i].strip().split())
print(score)

0


In [71]:
bu = sacrebleu.sentence_bleu(test['English'][1].strip(),[validation_data['English'][1]], smooth_method='exp')
print(bu.score)

2.874620762077756


In [ ]:

refs = []

with open("translated_validation.csv") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
print("Reference 1st sentence:", refs[0])

# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("target.pred") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)

# Calculate BLEU for sentence by sentence and save the result to a file
with open("bleu.txt", "w+") as output:
    for line in zip(refs,preds):
        test = line[0]
        pred = line[1]
        print(test, "\t--->\t", pred)
        bleu = sacrebleu.sentence_bleu(pred, [test], smooth_method='exp')
        print(bleu.score, "\n")
        output.write(str(bleu.score) + "\n")

In [47]:
translate(model, src=" tapas", max_len =40 , custom_sentence=True)

tensor([[4375]], device='cuda:1')


'<sos> she rose she came unto the so come hither her dear most dear thee come come come again come hither fire god brihaspati and savitar again dear would eat dear life again again etc'

In [40]:
translate(model, src=" yasmin devā amṛjata yasmin manuṣyā uta  tasmin ghṛtastāvo mṛṣṭvā tvam agne divaṃ ruha", max_len = 499, custom_sentence=True)

tensor([[ 1126,    53, 25120,  1126,  1771,    88,   880, 36041, 17327,    31,
            40,   851, 50732]], device='cuda:1')


'<sos> threefold are they met eternal sent eternal travel the rejoicing travel each wondrous creatures'

In [34]:
EN_TEXT.vocab.itos[1126]

'pathways'

In [ ]:
import bcolz
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'/home/deepak/Documents/Atharva/embeddings/san100.dat', mode='w')

with open(f'/home/deepak/Documents/Atharva/embeddings/vectors.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((6333000, 100)), rootdir=f'/home/deepak/Documents/Atharva/embeddings/san100.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'/home/deepak/Documents/Atharva/embeddings/san100_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'/home/deepak/Documents/Atharva/embeddings/san100_idx.pkl', 'wb'))